In [ ]:
from transformers import AutoModelForMaskedLM

In [ ]:
from utils import Metrics
gpt = Metrics.GPT2LM(cuda=1)
grammarchecker = Metrics.GrammarChecker()
use = Metrics.USE(1)
edit = Metrics.EditDistance()
edit_normalized = Metrics.EditDistance(normalized=True)

In [ ]:
import os
filenames = [k for n,j,k in os.walk('newFinal')][0]
print(filenames)

In [ ]:
from tqdm import tqdm
def read_log(path):
    ori, adv = [], []
    with open(path) as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith('ORI: '):
                ori.append(line[5:])
            elif line.startswith("ADV: "):
                adv.append(line[5:])
    print(f"ORI{len(ori)}, ADV{len(adv)}")
    return ori, adv

def eval_PPL(oris,advs, metric, uses=None, limit=0.0):
    print("Eval with" + str(metric))
    ori_PPLs = []
    adv_PPLs = []
    delta_PPLs = []
    length = len(oris)
    for i in tqdm(range(length)):
        if uses is not None and uses[i] < limit: continue
        ori, adv = oris[i], advs[i]
        ori_PPL = metric(ori)
        adv_PPL = metric(adv)
        delta_PPL = (adv_PPL - ori_PPL)
        
        ori_PPLs.append(ori_PPL)
        adv_PPLs.append(adv_PPL)
        delta_PPLs.append(delta_PPL)
    def mean(L):
        return sum(L)/ len(L)
    return  {
        'ori': mean(ori_PPLs), 
        'adv': mean(adv_PPLs), 
        'delta': mean(delta_PPLs), 
        'delta%': str(mean(delta_PPLs) / mean(ori_PPLs) * 100)[:7] + '%',
        }

def eval_score(oris, advs, metric):
    print("Eval with" + str(metric))
    scores = []
    length = len(oris)
    for i in tqdm(range(length)):
        ori, adv = oris[i], advs[i]
        score = metric(ori, adv)
        scores.append(score)
    def mean(L):
        return sum(L)/ len(L)
    return mean(scores), scores

def dict_putout(d):
    _d = {}
    for key in d.keys():
        if type(d[key]) != type({}):
            _d[key] = d[key]
        else:
            output_d = dict_putout(d[key])
            for _key in output_d.keys():
                _d[key + '_' + _key] = output_d[_key]
    return _d.copy()

eval_datas = {}
filenames = [
             '/home/phantivia/lab/PandRlib/SST2Exp/100-BertOnAlbert-SST2-Layer10-Adv5-Dw0.5-USE-FULL.out',
             '/home/phantivia/lab/PandRlib/SST2Exp/100-BertOnBert-SST2-Layer10-Adv5-Dw0.5-USE-FULL.out',
             '/home/phantivia/lab/PandRlib/SST2Exp/100-BertOnRoberta-SST2-Layer10-Adv5-Dw0.5-USE-FULL.out']
filenames = [
           '/home/phantivia/lab/PandRlib/SST2Exp/100-BertOnRoberta-SST2-Layer10-Adv5-Dw0.5-USE-FULL.out',
]

for file in filenames:
    if  'USE' in file:
        ori, adv = read_log(file)
        eval_datas[file] = {}
        
        eval_datas[file]['GrammarError'] = eval_PPL(ori, adv, grammarchecker)
        eval_datas[file]['USE'], uses = eval_score(ori, adv, use)
        eval_datas[file]['PPL'] = eval_PPL(ori, adv, gpt, uses, 0.7)
        eval_datas[file]['edit_distance'], _ = eval_score(ori, adv, edit)
        eval_datas[file]['edit_distance_normalized'], _ = eval_score(ori, adv, edit_normalized)
        
        eval_datas[file] = dict_putout(eval_datas[file])
        print(file, eval_datas[file])


In [ ]:
luses = [i for i in uses if i < 0.7]
len(luses)

In [ ]:
ruses = [i for i in uses if i >= 0.7]
sum(ruses) / len(ruses)

In [ ]:
ASR = (0.95 - (0.01 * 100 + len(luses))/100)/0.95
ASR

In [ ]:
units = []
for key in eval_datas.keys():
    units.append({
        'name':key,
        **eval_datas[key],
    })
import pandas
df = pandas.DataFrame(units)
df.to_csv('eval_result_100.csv')

In [ ]:
from scanf import scanf
pattern = 'BertOn%s-%s-Layer%s'

def s(d):
    a, b = scanf(pattern, d)
    return a, b
df = df.sort_values(by = ['victim', 'task'])
df = df[['victim', 'task','PPL_ori', 'PPL_adv', 'PPL_delta',
       'PPL_delta%', 'GrammarError_ori', 'GrammarError_adv',
       'GrammarError_delta', 'GrammarError_delta%', 'USE', 'edit_distance',
       'edit_distance_normalized']]
df

In [ ]:
import os
filenames = [k for n,j,k in os.walk('newFinal')][0]
print(filenames)
from scanf import scanf
pattern = 'BertOn%s-%s-Layer%s'
pattern0 = 'ASR=%f,BS=%f, %s'
pattern1 = '%sNUM_WORD_CHANGES=%f'
pattern2 = 'Query=%f, Bpsteps=%f, NumberEvalAttacks=%d, AverageStepInSeg=%f'

def get_rest(path):
    d = {}
    d['victim'], d['task'], _ = scanf(pattern, path)
    with open(path) as f:
        lines = f.readlines()
        for line in lines:
            args0 = scanf(pattern0, line)
            if args0 is not None:
                d['ASR'], d['BS'], _ = args0
                
            args1 = scanf(pattern1, line)
            if args1 is not None:
                _, d['NUM_WORD_CHANGES'] = args1
                
            args2 = scanf(pattern2, line)
            if args2 is not None:
                d['Query'], d['Bpsteps'], d['NumberEvalAttacks'], d['AverageStepInSeg'] = args2
    return d

datas = []
for file in filenames:
    if 'USE' in file:
        datas.append(get_rest('newFinal/' + file))
import pandas
rest_df = pandas.DataFrame(datas)
rest_df

In [ ]:
rest_df = rest_df.sort_values(by = ['victim', 'task'])
rest_df.to_csv('rest.csv')